<a href="https://colab.research.google.com/github/Bilal-Ahmad-5/Medical-Assistance/blob/main/Medical_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Healthcare AI Chatbot with LangGraph + Gemini 2.5 Flash
# Complete Implementation for Google Colab

# ============================================================================
# STEP 1: INSTALLATION AND IMPORTS
# ============================================================================

# Install required packages
!pip install langchain langchain-community langchain-google-genai
!pip install langgraph gradio
!pip install sentence-transformers faiss-cpu
!pip install pandas
!pip install pydantic

import sqlite3
import os
import json
import logging
from datetime import datetime, date, timedelta
from typing import Optional, List, Dict, Any, Annotated
from dataclasses import dataclass
import pandas as pd

# LangChain and LangGraph imports
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pydantic import BaseModel


# LangGraph imports
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Database configuration
DB_FILE = "healthcare_chatbot.db"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires req

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 61.0 MB/s eta 0:00:00


In [6]:
# ============================================================================
# SECTION 2: DATABASE SETUP AND SCHEMA
# ============================================================================

from langchain_core.tools import tool

def create_database():
    """Create the healthcare database with all necessary tables."""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    # Patients table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS patients (
        patient_id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        phone TEXT,
        date_of_birth DATE,
        insurance_info TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')

    # Doctors table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS doctors (
        doctor_id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        specialization TEXT NOT NULL,
        department TEXT,
        email TEXT,
        phone TEXT,
        available_days TEXT,  -- JSON string of available days
        available_hours TEXT, -- JSON string of available hours
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')

    # Appointments table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS appointments (
        appointment_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id INTEGER NOT NULL,
        doctor_id INTEGER NOT NULL,
        appointment_date DATE NOT NULL,
        appointment_time TIME NOT NULL,
        reason TEXT,
        status TEXT DEFAULT 'scheduled',  -- scheduled, completed, cancelled
        notes TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (patient_id) REFERENCES patients (patient_id),
        FOREIGN KEY (doctor_id) REFERENCES doctors (doctor_id)
    )
    ''')

    # Hospital services table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS services (
        service_id INTEGER PRIMARY KEY AUTOINCREMENT,
        service_name TEXT NOT NULL,
        department TEXT,
        description TEXT,
        cost REAL,
        duration_minutes INTEGER
    )
    ''')

    conn.commit()
    conn.close()
    print("Database created successfully!")

def populate_sample_data():
    """Populate the database with comprehensive sample data."""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    # Extended sample patients (20 patients)
    patients = [
        ('John', 'Doe', 'john.doe@email.com', '+1234567890', '1985-05-15', 'Blue Cross Blue Shield'),
        ('Jane', 'Smith', 'jane.smith@email.com', '+1234567891', '1990-08-22', 'Aetna'),
        ('Mike', 'Johnson', 'mike.johnson@email.com', '+1234567892', '1978-12-10', 'Medicare'),
        ('Sarah', 'Wilson', 'sarah.wilson@email.com', '+1234567893', '1995-03-08', 'Cigna'),
        ('David', 'Brown', 'david.brown@email.com', '+1234567894', '1982-11-30', 'United Healthcare'),
        ('Emma', 'Davis', 'emma.davis@email.com', '+1234567895', '1988-09-14', 'Blue Cross Blue Shield'),
        ('Chris', 'Miller', 'chris.miller@email.com', '+1234567896', '1975-07-02', 'Medicare'),
        ('Lisa', 'Garcia', 'lisa.garcia@email.com', '+1234567897', '1992-04-18', 'Aetna'),
        ('Robert', 'Martinez', 'robert.martinez@email.com', '+1234567898', '1980-12-25', 'Cigna'),
        ('Amanda', 'Rodriguez', 'amanda.rodriguez@email.com', '+1234567899', '1987-06-08', 'United Healthcare'),
        ('Kevin', 'Lee', 'kevin.lee@email.com', '+1234567810', '1991-01-12', 'Blue Cross Blue Shield'),
        ('Maria', 'Gonzalez', 'maria.gonzalez@email.com', '+1234567811', '1983-10-05', 'Medicaid'),
        ('James', 'White', 'james.white@email.com', '+1234567812', '1979-03-20', 'Aetna'),
        ('Ashley', 'Thomas', 'ashley.thomas@email.com', '+1234567813', '1994-08-11', 'Cigna'),
        ('Michael', 'Taylor', 'michael.taylor@email.com', '+1234567814', '1986-02-28', 'United Healthcare'),
        ('Jessica', 'Anderson', 'jessica.anderson@email.com', '+1234567815', '1989-11-17', 'Blue Cross Blue Shield'),
        ('Daniel', 'Jackson', 'daniel.jackson@email.com', '+1234567816', '1977-05-03', 'Medicare'),
        ('Nicole', 'Harris', 'nicole.harris@email.com', '+1234567817', '1993-09-22', 'Aetna'),
        ('Ryan', 'Clark', 'ryan.clark@email.com', '+1234567818', '1984-07-15', 'Cigna'),
        ('Stephanie', 'Lewis', 'stephanie.lewis@email.com', '+1234567819', '1996-12-07', 'Medicaid'),
    ]

    cursor.executemany('''
    INSERT OR IGNORE INTO patients (first_name, last_name, email, phone, date_of_birth, insurance_info)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', patients)

    # Extended sample doctors (15 doctors)
    doctors = [
        ('Dr. Emily', 'Anderson', 'Cardiology', 'Heart Center', 'emily.anderson@hospital.com', '+1234567800',
         '["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]', '["09:00", "10:00", "11:00", "14:00", "15:00", "16:00"]'),
        ('Dr. Michael', 'Brown', 'Orthopedics', 'Bone & Joint Center', 'michael.brown@hospital.com', '+1234567801',
         '["Monday", "Wednesday", "Friday"]', '["08:00", "09:00", "10:00", "11:00", "13:00", "14:00"]'),
        ('Dr. Lisa', 'Davis', 'Pediatrics', 'Children\'s Center', 'lisa.davis@hospital.com', '+1234567802',
         '["Tuesday", "Thursday", "Friday"]', '["09:00", "10:00", "11:00", "13:00", "14:00", "15:00"]'),
        ('Dr. Robert', 'Miller', 'Internal Medicine', 'General Medicine', 'robert.miller@hospital.com', '+1234567803',
         '["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]', '["08:00", "09:00", "10:00", "14:00", "15:00", "16:00"]'),
        ('Dr. Jennifer', 'Taylor', 'Dermatology', 'Skin Care Center', 'jennifer.taylor@hospital.com', '+1234567804',
         '["Monday", "Tuesday", "Thursday"]', '["10:00", "11:00", "14:00", "15:00", "16:00"]'),
        ('Dr. William', 'Johnson', 'Neurology', 'Brain & Spine Center', 'william.johnson@hospital.com', '+1234567805',
         '["Tuesday", "Wednesday", "Thursday"]', '["09:00", "10:00", "11:00", "14:00", "15:00"]'),
        ('Dr. Sarah', 'Wilson', 'Gynecology', 'Women\'s Health Center', 'sarah.wilson@hospital.com', '+1234567806',
         '["Monday", "Wednesday", "Friday"]', '["08:00", "09:00", "10:00", "13:00", "14:00", "15:00"]'),
        ('Dr. Thomas', 'Garcia', 'Psychiatry', 'Mental Health Center', 'thomas.garcia@hospital.com', '+1234567807',
         '["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]', '["10:00", "11:00", "14:00", "15:00", "16:00"]'),
        ('Dr. Patricia', 'Martinez', 'Oncology', 'Cancer Treatment Center', 'patricia.martinez@hospital.com', '+1234567808',
         '["Monday", "Tuesday", "Wednesday", "Thursday"]', '["09:00", "10:00", "11:00", "13:00", "14:00"]'),
        ('Dr. Christopher', 'Lee', 'Ophthalmology', 'Eye Care Center', 'christopher.lee@hospital.com', '+1234567809',
         '["Tuesday", "Thursday", "Friday"]', '["08:00", "09:00", "10:00", "14:00", "15:00"]'),
        ('Dr. Michelle', 'Rodriguez', 'Endocrinology', 'Diabetes & Hormone Center', 'michelle.rodriguez@hospital.com', '+1234567820',
         '["Monday", "Wednesday", "Thursday"]', '["09:00", "10:00", "11:00", "14:00", "15:00"]'),
        ('Dr. Andrew', 'White', 'Urology', 'Kidney & Bladder Center', 'andrew.white@hospital.com', '+1234567821',
         '["Monday", "Tuesday", "Friday"]', '["08:00", "09:00", "10:00", "13:00", "14:00"]'),
        ('Dr. Karen', 'Thomas', 'Rheumatology', 'Joint & Arthritis Center', 'karen.thomas@hospital.com', '+1234567822',
         '["Tuesday", "Wednesday", "Thursday"]', '["10:00", "11:00", "14:00", "15:00", "16:00"]'),
        ('Dr. Steven', 'Jackson', 'Pulmonology', 'Lung & Respiratory Center', 'steven.jackson@hospital.com', '+1234567823',
         '["Monday", "Wednesday", "Friday"]', '["09:00", "10:00", "11:00", "13:00", "14:00"]'),
        ('Dr. Linda', 'Harris', 'Gastroenterology', 'Digestive Health Center', 'linda.harris@hospital.com', '+1234567824',
         '["Tuesday", "Thursday", "Friday"]', '["08:00", "09:00", "10:00", "14:00", "15:00"]'),
    ]

    cursor.executemany('''
    INSERT OR IGNORE INTO doctors (first_name, last_name, specialization, department, email, phone, available_days, available_hours)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', doctors)

    # Extended sample services (25 services)
    services = [
        ('General Consultation', 'General Medicine', 'Basic medical consultation and examination', 150.0, 30),
        ('Cardiology Consultation', 'Cardiology', 'Heart and cardiovascular system examination', 300.0, 45),
        ('X-Ray', 'Radiology', 'Digital X-ray imaging service', 100.0, 15),
        ('Blood Test', 'Laboratory', 'Complete blood count and basic metabolic panel', 80.0, 10),
        ('MRI Scan', 'Radiology', 'Magnetic resonance imaging', 800.0, 60),
        ('Physical Therapy', 'Rehabilitation', 'Physical therapy session', 120.0, 60),
        ('Vaccination', 'Preventive Care', 'Various vaccination services', 50.0, 15),
        ('CT Scan', 'Radiology', 'Computed tomography scan', 600.0, 45),
        ('Ultrasound', 'Radiology', 'Ultrasound imaging examination', 200.0, 30),
        ('ECG/EKG', 'Cardiology', 'Electrocardiogram heart rhythm test', 100.0, 15),
        ('Echocardiogram', 'Cardiology', 'Heart ultrasound examination', 350.0, 45),
        ('Mammography', 'Radiology', 'Breast cancer screening', 250.0, 20),
        ('Colonoscopy', 'Gastroenterology', 'Colon and rectum examination', 900.0, 90),
        ('Endoscopy', 'Gastroenterology', 'Upper digestive tract examination', 700.0, 60),
        ('Bone Density Test', 'Orthopedics', 'Osteoporosis screening test', 180.0, 30),
        ('Stress Test', 'Cardiology', 'Cardiac stress testing', 400.0, 60),
        ('Spirometry', 'Pulmonology', 'Lung function test', 150.0, 30),
        ('Allergy Testing', 'Immunology', 'Comprehensive allergy panel', 300.0, 60),
        ('Sleep Study', 'Sleep Medicine', 'Overnight sleep disorder evaluation', 1200.0, 480),
        ('Dermatology Screening', 'Dermatology', 'Skin cancer screening examination', 200.0, 30),
        ('Eye Examination', 'Ophthalmology', 'Comprehensive eye and vision test', 180.0, 45),
        ('Hearing Test', 'Audiology', 'Complete hearing evaluation', 150.0, 60),
        ('Mental Health Assessment', 'Psychiatry', 'Psychological evaluation and consultation', 250.0, 90),
        ('Nutrition Consultation', 'Nutrition', 'Dietary assessment and planning', 120.0, 45),
        ('Diabetes Management', 'Endocrinology', 'Comprehensive diabetes care consultation', 200.0, 60),
    ]

    cursor.executemany('''
    INSERT OR IGNORE INTO services (service_name, department, description, cost, duration_minutes)
    VALUES (?, ?, ?, ?, ?)
    ''', services)

    # Extended sample appointments (30 appointments with varied dates and statuses)
    appointments = [
        # Past appointments (completed/cancelled)
        (1, 1, '2024-01-15', '09:00:00', 'Chest pain consultation', 'completed'),
        (2, 2, '2024-01-16', '10:00:00', 'Knee pain examination', 'completed'),
        (3, 3, '2024-01-17', '14:00:00', 'Child wellness check', 'completed'),
        (1, 4, '2024-01-18', '15:00:00', 'Annual physical', 'completed'),
        (5, 5, '2024-01-19', '11:00:00', 'Skin rash consultation', 'cancelled'),

        # Current/upcoming appointments (scheduled)
        (4, 1, '2024-12-15', '10:00:00', 'Follow-up cardiac consultation', 'scheduled'),
        (6, 6, '2024-12-16', '09:00:00', 'Headache evaluation', 'scheduled'),
        (7, 7, '2024-12-17', '14:00:00', 'Women\'s health checkup', 'scheduled'),
        (8, 8, '2024-12-18', '15:00:00', 'Anxiety counseling session', 'scheduled'),
        (9, 9, '2024-12-19', '10:00:00', 'Cancer screening consultation', 'scheduled'),
        (10, 10, '2024-12-20', '11:00:00', 'Eye examination', 'scheduled'),
        (11, 11, '2024-12-21', '09:00:00', 'Diabetes management', 'scheduled'),
        (12, 12, '2024-12-22', '14:00:00', 'Arthritis consultation', 'scheduled'),
        (13, 13, '2024-12-23', '10:00:00', 'Breathing difficulties', 'scheduled'),
        (14, 14, '2024-12-24', '15:00:00', 'Stomach pain evaluation', 'scheduled'),
        (15, 1, '2024-12-26', '09:00:00', 'Heart palpitations', 'scheduled'),
        (16, 2, '2024-12-27', '10:00:00', 'Hip replacement consultation', 'scheduled'),
        (17, 3, '2024-12-28', '11:00:00', 'Child vaccination', 'scheduled'),
        (18, 4, '2024-12-30', '14:00:00', 'Hypertension management', 'scheduled'),
        (19, 5, '2025-01-02', '15:00:00', 'Mole examination', 'scheduled'),
        (20, 6, '2025-01-03', '09:00:00', 'Migraine treatment', 'scheduled'),

        # Future appointments
        (1, 7, '2025-01-05', '10:00:00', 'Routine gynecological exam', 'scheduled'),
        (2, 8, '2025-01-06', '11:00:00', 'Depression therapy session', 'scheduled'),
        (3, 9, '2025-01-08', '14:00:00', 'Oncology follow-up', 'scheduled'),
        (4, 10, '2025-01-09', '15:00:00', 'Vision problems', 'scheduled'),
        (5, 11, '2025-01-10', '09:00:00', 'Thyroid disorder consultation', 'scheduled'),
        (6, 12, '2025-01-12', '10:00:00', 'Kidney stone treatment', 'scheduled'),
        (7, 13, '2025-01-13', '11:00:00', 'Joint pain assessment', 'scheduled'),
        (8, 14, '2025-01-15', '14:00:00', 'Chronic cough evaluation', 'scheduled'),
        (9, 15, '2025-01-16', '15:00:00', 'Digestive issues consultation', 'scheduled'),
        (10, 1, '2025-01-17', '09:00:00', 'Cardiac stress test', 'scheduled'),
    ]

    cursor.executemany('''
    INSERT OR IGNORE INTO appointments (patient_id, doctor_id, appointment_date, appointment_time, reason, status)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', appointments)

    conn.commit()
    cursor.close()
    conn.close()
    print("Comprehensive sample data populated successfully!")
    print("📊 Data Summary:")
    print("  • 20 Patients")
    print("  • 15 Doctors across 15 specializations")
    print("  • 25 Hospital services")
    print("  • 30 Appointments (past, current, and future)")

In [7]:
# ============================================================================
# STEP 4: SMART DATABASE TOOLS FOR LANGGRAPH
# ============================================================================

@tool
def search_appointments(
    patient_id: Optional[int] = None,
    patient_name: Optional[str] = None,
    doctor_name: Optional[str] = None,
    specialization: Optional[str] = None,
    date: Optional[str] = None,
    status: Optional[str] = None,
    limit: int = 20,
) -> Dict[str, Any]:
    """
    Smart appointment search with intelligent filtering.
    Returns structured data for LangGraph workflow decisions.
    """
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()

        query = '''
        SELECT a.appointment_id, a.patient_id, a.doctor_id, a.appointment_date,
               a.appointment_time, a.reason, a.status, a.notes,
               p.first_name || ' ' || p.last_name as patient_name,
               d.first_name || ' ' || d.last_name as doctor_name,
               d.specialization, d.department
        FROM appointments a
        JOIN patients p ON a.patient_id = p.patient_id
        JOIN doctors d ON a.doctor_id = d.doctor_id
        WHERE 1 = 1
        '''
        params = []
        search_context = []

        if patient_id:
            query += " AND a.patient_id = ?"
            params.append(patient_id)
            search_context.append(f"Patient ID: {patient_id}")

        if patient_name:
            query += " AND (p.first_name LIKE ? OR p.last_name LIKE ? OR (p.first_name || ' ' || p.last_name) LIKE ?)"
            params.extend([f"%{patient_name}%", f"%{patient_name}%", f"%{patient_name}%"])
            search_context.append(f"Patient: {patient_name}")

        if doctor_name:
            query += " AND (d.first_name LIKE ? OR d.last_name LIKE ? OR (d.first_name || ' ' || d.last_name) LIKE ?)"
            params.extend([f"%{doctor_name}%", f"%{doctor_name}%", f"%{doctor_name}%"])
            search_context.append(f"Doctor: {doctor_name}")

        if specialization:
            query += " AND d.specialization LIKE ?"
            params.append(f"%{specialization}%")
            search_context.append(f"Specialization: {specialization}")

        if date:
            query += " AND a.appointment_date = ?"
            params.append(date)
            search_context.append(f"Date: {date}")

        if status:
            query += " AND a.status = ?"
            params.append(status)
            search_context.append(f"Status: {status}")

        query += " ORDER BY a.appointment_date DESC, a.appointment_time DESC LIMIT ?"
        params.append(limit)

        cursor.execute(query, params)
        rows = cursor.fetchall()

        appointments = []
        for row in rows:
            appointment = {
                "appointment_id": row[0],
                "patient_id": row[1],
                "doctor_id": row[2],
                "date": row[3],
                "time": row[4],
                "reason": row[5],
                "status": row[6],
                "notes": row[7] or "",
                "patient_name": row[8],
                "doctor_name": row[9],
                "specialization": row[10],
                "department": row[11],
                "is_upcoming": row[3] >= datetime.now().strftime('%Y-%m-%d') and row[6] == 'scheduled'
            }
            appointments.append(appointment)

        cursor.close()
        conn.close()

        return {
            "success": True,
            "appointments": appointments,
            "count": len(appointments),
            "search_criteria": search_context,
            "message": f"Found {len(appointments)} appointment(s)"
        }

    except Exception as e:
        return {
            "success": False,
            "appointments": [],
            "count": 0,
            "errors": [f"Error: {str(e)}"],
            "message": "Search failed"
        }

@tool
def book_appointment(
    patient_id: int,
    doctor_id: int,
    appointment_date: str,
    appointment_time: str,
    reason: str,
    notes: Optional[str] = None,
) -> Dict[str, Any]:
    """
    Smart appointment booking with comprehensive validation.
    Returns detailed booking confirmation for LangGraph workflow.
    """
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()

        # Validate patient exists
        cursor.execute("SELECT first_name, last_name FROM patients WHERE patient_id = ?", (patient_id,))
        patient = cursor.fetchone()
        if not patient:
            cursor.close()
            conn.close()
            return {
                "success": False,
                "appointment_id": None,
                "errors": [f"Patient ID {patient_id} not found"],
                "message": "Booking failed - patient not found"
            }

        # Validate doctor exists
        cursor.execute("SELECT first_name, last_name, specialization, available_days, available_hours FROM doctors WHERE doctor_id = ?", (doctor_id,))
        doctor = cursor.fetchone()
        if not doctor:
            cursor.close()
            conn.close()
            return {
                "success": False,
                "appointment_id": None,
                "errors": [f"Doctor ID {doctor_id} not found"],
                "message": "Booking failed - doctor not found"
            }

        # Validate date/time format and check if it's in the future
        try:
            appointment_datetime = datetime.strptime(f"{appointment_date} {appointment_time}", '%Y-%m-%d %H:%M')
            if appointment_datetime < datetime.now():
                cursor.close()
                conn.close()
                return {
                    "success": False,
                    "appointment_id": None,
                    "errors": ["Cannot book appointments in the past"],
                    "message": "Booking failed - invalid date/time"
                }
        except ValueError:
            cursor.close()
            conn.close()
            return {
                "success": False,
                "appointment_id": None,
                "errors": ["Invalid date/time format. Use YYYY-MM-DD and HH:MM"],
                "message": "Booking failed - invalid format"
            }

        # Check doctor availability
        try:
            available_days = json.loads(doctor[3])
            available_hours = json.loads(doctor[4])
            day_name = appointment_datetime.strftime('%A')

            if day_name not in available_days or appointment_time not in available_hours:
                cursor.close()
                conn.close()
                return {
                    "success": False,
                    "appointment_id": None,
                    "errors": [f"Dr. {doctor[0]} {doctor[1]} is not available at this time"],
                    "suggestions": [f"Available days: {', '.join(available_days)}", f"Available hours: {', '.join(available_hours)}"],
                    "message": "Booking failed - doctor not available"
                }
        except (json.JSONDecodeError, ValueError):
            pass  # Continue with booking if availability check fails

        # Check for time conflicts
        cursor.execute('''
        SELECT appointment_id FROM appointments
        WHERE doctor_id = ? AND appointment_date = ? AND appointment_time = ? AND status != 'cancelled'
        ''', (doctor_id, appointment_date, appointment_time))

        if cursor.fetchone():
            cursor.close()
            conn.close()
            return {
                "success": False,
                "appointment_id": None,
                "errors": ["Time slot already booked"],
                "message": "Booking failed - time slot conflict"
            }

        # Book the appointment
        cursor.execute('''
        INSERT INTO appointments (patient_id, doctor_id, appointment_date, appointment_time, reason, notes, status)
        VALUES (?, ?, ?, ?, ?, ?, 'scheduled')
        ''', (patient_id, doctor_id, appointment_date, appointment_time, reason, notes))

        appointment_id = cursor.lastrowid
        conn.commit()
        cursor.close()
        conn.close()

        return {
            "success": True,
            "appointment_id": appointment_id,
            "booking_details": {
                "appointment_id": appointment_id,
                "patient_name": f"{patient[0]} {patient[1]}",
                "doctor_name": f"Dr. {doctor[0]} {doctor[1]}",
                "specialization": doctor[2],
                "date": appointment_date,
                "time": appointment_time,
                "reason": reason,
                "status": "scheduled"
            },
            "message": f"Appointment successfully booked! ID: {appointment_id}",
            "confirmation": f"Confirmed: {patient[0]} {patient[1]} with Dr. {doctor[0]} {doctor[1]} ({doctor[2]}) on {appointment_date} at {appointment_time}"
        }

    except Exception as e:
        return {
            "success": False,
            "appointment_id": None,
            "errors": [f"Error: {str(e)}"],
            "message": "Booking failed due to error"
        }

@tool
def update_appointment(
    appointment_id: int,
    new_date: Optional[str] = None,
    new_time: Optional[str] = None,
    new_reason: Optional[str] = None,
    new_notes: Optional[str] = None,
) -> Dict[str, Any]:
    """
    Smart appointment update with conflict detection and validation.
    Returns detailed update confirmation for LangGraph workflow.
    """
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()

        # Get current appointment details
        cursor.execute('''
        SELECT a.appointment_id, a.patient_id, a.doctor_id, a.appointment_date,
               a.appointment_time, a.reason, a.status, a.notes,
               p.first_name || ' ' || p.last_name as patient_name,
               d.first_name || ' ' || d.last_name as doctor_name,
               d.specialization
        FROM appointments a
        JOIN patients p ON a.patient_id = p.patient_id
        JOIN doctors d ON a.doctor_id = d.doctor_id
        WHERE a.appointment_id = ?
        ''', (appointment_id,))

        appointment = cursor.fetchone()
        if not appointment:
            cursor.close()
            conn.close()
            return {
                "success": False,
                "errors": [f"Appointment ID {appointment_id} not found"],
                "message": "Update failed - appointment not found"
            }

        current_status = appointment[6]
        if current_status in ['cancelled', 'completed']:
            cursor.close()
            conn.close()
            return {
                "success": False,
                "errors": [f"Cannot update a {current_status} appointment"],
                "message": f"Update failed - appointment is {current_status}"
            }

        # Prepare update query
        updates = []
        params = []
        changes = []

        if new_date:
            updates.append("appointment_date = ?")
            params.append(new_date)
            changes.append(f"Date: {appointment[3]} → {new_date}")

        if new_time:
            updates.append("appointment_time = ?")
            params.append(new_time)
            changes.append(f"Time: {appointment[4]} → {new_time}")

        if new_reason:
            updates.append("reason = ?")
            params.append(new_reason)
            changes.append(f"Reason updated")

        if new_notes is not None:
            updates.append("notes = ?")
            params.append(new_notes)
            changes.append("Notes updated")

        if not updates:
            cursor.close()
            conn.close()
            return {
                "success": False,
                "errors": ["No changes specified"],
                "message": "Update failed - no changes provided"
            }

        # Perform update
        updates.append("updated_at = CURRENT_TIMESTAMP")
        params.append(appointment_id)

        query = f"UPDATE appointments SET {', '.join(updates)} WHERE appointment_id = ?"
        cursor.execute(query, params)
        conn.commit()
        cursor.close()
        conn.close()

        return {
            "success": True,
            "appointment_id": appointment_id,
            "changes_made": changes,
            "updated_appointment": {
                "appointment_id": appointment_id,
                "patient_name": appointment[8],
                "doctor_name": appointment[9],
                "specialization": appointment[10],
                "date": new_date or appointment[3],
                "time": new_time or appointment[4],
                "reason": new_reason or appointment[5]
            },
            "message": f"Appointment {appointment_id} updated successfully",
            "confirmation": f"Updated: {appointment[8]} with {appointment[9]} - Changes: {', '.join(changes)}"
        }

    except Exception as e:
        return {
            "success": False,
            "errors": [f"Error: {str(e)}"],
            "message": "Update failed due to error"
        }

@tool
def cancel_appointment(
    appointment_id: int,
    cancellation_reason: Optional[str] = None,
) -> Dict[str, Any]:
    """
    Smart appointment cancellation with rebooking suggestions.
    Returns detailed cancellation confirmation for LangGraph workflow.
    """
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()

        # Get complete appointment details
        cursor.execute('''
        SELECT a.appointment_id, a.patient_id, a.doctor_id, a.appointment_date,
               a.appointment_time, a.reason, a.status, a.notes,
               p.first_name || ' ' || p.last_name as patient_name,
               d.first_name || ' ' || d.last_name as doctor_name,
               d.specialization, d.department
        FROM appointments a
        JOIN patients p ON a.patient_id = p.patient_id
        JOIN doctors d ON a.doctor_id = d.doctor_id
        WHERE a.appointment_id = ?
        ''', (appointment_id,))

        appointment = cursor.fetchone()
        if not appointment:
            cursor.close()
            conn.close()
            return {
                "success": False,
                "errors": [f"Appointment ID {appointment_id} not found"],
                "message": "Cancellation failed - appointment not found"
            }

        current_status = appointment[6]
        if current_status == 'cancelled':
            cursor.close()
            conn.close()
            return {
                "success": False,
                "errors": ["Appointment is already cancelled"],
                "message": "Cancellation not needed - appointment already cancelled"
            }

        if current_status == 'completed':
            cursor.close()
            conn.close()
            return {
                "success": False,
                "errors": ["Cannot cancel a completed appointment"],
                "message": "Cancellation failed - appointment already completed"
            }

        # Prepare notes with cancellation reason
        updated_notes = appointment[7] or ""
        if cancellation_reason:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")
            cancellation_note = f"\n[{timestamp}] CANCELLED: {cancellation_reason}"
            updated_notes = (updated_notes + cancellation_note).strip()

        # Perform cancellation
        cursor.execute('''
        UPDATE appointments
        SET status = 'cancelled', notes = ?, updated_at = CURRENT_TIMESTAMP
        WHERE appointment_id = ?
        ''', (updated_notes, appointment_id))

        conn.commit()
        cursor.close()
        conn.close()

        return {
            "success": True,
            "appointment_id": appointment_id,
            "cancelled_appointment": {
                "appointment_id": appointment_id,
                "patient_name": appointment[8],
                "doctor_name": appointment[9],
                "specialization": appointment[10],
                "date": appointment[3],
                "time": appointment[4],
                "original_reason": appointment[5],
                "cancellation_reason": cancellation_reason
            },
            "message": f"Appointment {appointment_id} cancelled successfully",
            "confirmation": f"Cancelled: {appointment[8]}'s appointment with {appointment[9]} on {appointment[3]} at {appointment[4]}"
        }

    except Exception as e:
        return {
            "success": False,
            "errors": [f"Error: {str(e)}"],
            "message": "Cancellation failed due to error"
        }

@tool
def search_doctors(
    specialization: Optional[str] = None,
    name: Optional[str] = None,
    available_date: Optional[str] = None,
    limit: int = 10,
) -> Dict[str, Any]:
    """
    Smart doctor search with availability checking.
    Returns structured doctor data for LangGraph workflow decisions.
    """
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()

        query = '''
        SELECT doctor_id, first_name, last_name, specialization, department,
               email, phone, available_days, available_hours
        FROM doctors WHERE 1 = 1
        '''
        params = []
        search_context = []

        if specialization:
            query += " AND specialization LIKE ?"
            params.append(f"%{specialization}%")
            search_context.append(f"Specialization: {specialization}")

        if name:
            query += " AND (first_name LIKE ? OR last_name LIKE ? OR (first_name || ' ' || last_name) LIKE ?)"
            params.extend([f"%{name}%", f"%{name}%", f"%{name}%"])
            search_context.append(f"Name: {name}")

        query += " ORDER BY specialization, last_name LIMIT ?"
        params.append(limit)

        cursor.execute(query, params)
        rows = cursor.fetchall()

        doctors = []
        for row in rows:
            doctor = {
                "doctor_id": row[0],
                "first_name": row[1],
                "last_name": row[2],
                "full_name": f"Dr. {row[1]} {row[2]}",
                "specialization": row[3],
                "department": row[4],
                "email": row[5],
                "phone": row[6],
                "available_days": row[7],
                "available_hours": row[8]
            }

            # Add availability info for specific date if requested
            if available_date:
                try:
                    date_obj = datetime.strptime(available_date, '%Y-%m-%d')
                    day_name = date_obj.strftime('%A')
                    available_days_list = json.loads(row[7])
                    available_hours_list = json.loads(row[8])

                    doctor['is_available_on_date'] = day_name in available_days_list

                    if doctor['is_available_on_date']:
                        # Check actual free slots
                        cursor.execute('''
                        SELECT appointment_time FROM appointments
                        WHERE doctor_id = ? AND appointment_date = ? AND status != 'cancelled'
                        ''', (row[0], available_date))

                        booked_times = [apt_row[0] for apt_row in cursor.fetchall()]
                        free_slots = [time for time in available_hours_list if time not in booked_times]

                        doctor['available_slots'] = free_slots
                        doctor['total_free_slots'] = len(free_slots)
                    else:
                        doctor['available_slots'] = []
                        doctor['total_free_slots'] = 0

                except (ValueError, json.JSONDecodeError):
                    doctor['is_available_on_date'] = None
                    doctor['available_slots'] = []
                    doctor['total_free_slots'] = 0

            doctors.append(doctor)

        cursor.close()
        conn.close()

        return {
            "success": True,
            "doctors": doctors,
            "count": len(doctors),
            "search_criteria": search_context,
            "message": f"Found {len(doctors)} doctor(s)"
        }

    except Exception as e:
        return {
            "success": False,
            "doctors": [],
            "count": 0,
            "errors": [f"Error: {str(e)}"],
            "message": "Doctor search failed"
        }

@tool
def get_patient_info(patient_id: int) -> Dict[str, Any]:
    """
    Smart patient lookup with appointment history.
    Returns comprehensive patient data for LangGraph workflow context.
    """
    try:
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()

        # Get patient basic info
        cursor.execute("SELECT * FROM patients WHERE patient_id = ?", (patient_id,))
        patient_row = cursor.fetchone()

        if not patient_row:
            cursor.close()
            conn.close()
            return {
                "success": False,
                "errors": [f"Patient ID {patient_id} not found"],
                "patient_info": None,
                "message": "Patient lookup failed - ID not found"
            }

        # Build patient info
        patient_info = {
            "patient_id": patient_row[0],
            "first_name": patient_row[1],
            "last_name": patient_row[2],
            "full_name": f"{patient_row[1]} {patient_row[2]}",
            "email": patient_row[3],
            "phone": patient_row[4],
            "date_of_birth": patient_row[5],
            "insurance_info": patient_row[6]
        }

        # Get appointment statistics
        cursor.execute('''
        SELECT
            COUNT(*) as total,
            COUNT(CASE WHEN status = 'scheduled' THEN 1 END) as scheduled,
            COUNT(CASE WHEN status = 'completed' THEN 1 END) as completed,
            COUNT(CASE WHEN status = 'cancelled' THEN 1 END) as cancelled
        FROM appointments WHERE patient_id = ?
        ''', (patient_id,))

        stats = cursor.fetchone()
        appointment_stats = {
            "total_appointments": stats[0] if stats[0] else 0,
            "scheduled": stats[1] if stats[1] else 0,
            "completed": stats[2] if stats[2] else 0,
            "cancelled": stats[3] if stats[3] else 0
        }

        # Get upcoming appointments
        cursor.execute('''
        SELECT a.appointment_id, a.appointment_date, a.appointment_time, a.reason,
               d.first_name || ' ' || d.last_name as doctor_name, d.specialization
        FROM appointments a
        JOIN doctors d ON a.doctor_id = d.doctor_id
        WHERE a.patient_id = ? AND a.status = 'scheduled'
        AND a.appointment_date >= date('now')
        ORDER BY a.appointment_date, a.appointment_time
        LIMIT 5
        ''', (patient_id,))

        upcoming = []
        for row in cursor.fetchall():
            upcoming.append({
                "appointment_id": row[0],
                "date": row[1],
                "time": row[2],
                "reason": row[3],
                "doctor_name": row[4],
                "specialization": row[5]
            })

        cursor.close()
        conn.close()

        return {
            "success": True,
            "patient_info": patient_info,
            "appointment_statistics": appointment_stats,
            "upcoming_appointments": upcoming,
            "message": f"Retrieved information for {patient_info['full_name']}"
        }

    except Exception as e:
        return {
            "success": False,
            "errors": [f"Error: {str(e)}"],
            "patient_info": None,
            "message": "Patient lookup failed"
        }


In [8]:
# ============================================================================
# STEP 5: RAG KNOWLEDGE BASE SETUP
# ============================================================================

def create_knowledge_base():
    """Create a comprehensive knowledge base with detailed hospital information."""
    hospital_docs = [
        {
            "content": """
            Hospital Services and Departments:

            1. Emergency Department: Open 24/7 for urgent medical care, trauma center level II,
               equipped with advanced life support, helicopter landing pad
            2. Cardiology Department: Heart and cardiovascular care, Mon-Fri 8AM-5PM,
               cardiac catheterization lab, echocardiography, stress testing, pacemaker clinic
            3. Orthopedics Department: Bone, joint, and muscle care, specializes in sports injuries,
               joint replacement, spine surgery, arthroscopic procedures
            4. Pediatrics Department: Children's healthcare from birth to 18 years,
               NICU, pediatric surgery, child psychology, vaccination programs
            5. Internal Medicine: General adult healthcare and preventive medicine,
               chronic disease management, wellness programs, executive physicals
            6. Dermatology: Skin, hair, and nail conditions, cosmetic procedures,
               skin cancer screening, Mohs surgery, laser treatments
            7. Radiology: X-rays, MRI, CT scans, ultrasounds, mammography,
               nuclear medicine, interventional radiology, 3D imaging
            8. Laboratory Services: Blood tests, urine analysis, diagnostic tests,
               pathology, microbiology, genetic testing, 24/7 emergency lab
            9. Physical Therapy: Rehabilitation and recovery services,
               occupational therapy, speech therapy, sports medicine rehab
            10. Pharmacy: On-site prescription filling and medication counseling,
                specialty medications, clinical pharmacy services, medication therapy management
            """,
            "metadata": {"source": "hospital_services", "category": "services"}
        },
        {
            "content": """
            Specialized Medical Centers and Advanced Services:

            1. Cancer Treatment Center: Oncology, hematology, radiation therapy, chemotherapy,
               immunotherapy, clinical trials, genetic counseling, survivorship programs
            2. Women's Health Center: Obstetrics, gynecology, maternal-fetal medicine,
               fertility services, mammography, bone density testing, menopause clinic
            3. Heart and Vascular Institute: Interventional cardiology, cardiac surgery,
               vascular surgery, heart failure clinic, arrhythmia management, cardiac rehabilitation
            4. Neuroscience Center: Neurology, neurosurgery, stroke center, epilepsy clinic,
               movement disorders, brain tumor treatment, spinal cord injury rehabilitation
            5. Digestive Health Center: Gastroenterology, hepatology, endoscopy,
               colonoscopy, liver disease treatment, inflammatory bowel disease clinic
            6. Respiratory Care Center: Pulmonology, sleep medicine, lung transplant evaluation,
               COPD management, asthma clinic, pulmonary rehabilitation
            7. Mental Health Services: Psychiatry, psychology, addiction treatment,
               crisis intervention, group therapy, family counseling, adolescent programs
            8. Surgical Services: General surgery, minimally invasive surgery, robotic surgery,
               same-day surgery, pre-operative evaluation, post-operative care
            """,
            "metadata": {"source": "specialized_centers", "category": "services"}
        },
        {
            "content": """
            Insurance and Billing Information:

            Accepted Insurance Plans:
            - Blue Cross Blue Shield (all plans), Aetna, Cigna, United Healthcare
            - Medicare (Parts A, B, C, D), Medicaid, TriCare, Workers' Compensation
            - Self-pay options available with sliding scale fees based on income

            Payment Options:
            - Cash, all major credit/debit cards, personal checks, money orders
            - Payment plans available for procedures over $500 (0% interest for 12 months)
            - CareCredit healthcare financing accepted
            - HSA and FSA cards accepted

            Billing Process:
            - Bills sent within 30 days of service
            - Insurance claims filed automatically within 24 hours
            - Patient responsibility determined after insurance processing
            - Financial assistance programs available for qualifying patients (up to 100% discount)
            - Billing questions answered Monday-Friday 8AM-6PM
            - Online patient portal for viewing and paying bills
            - Automatic payment plans available

            Cost Transparency:
            - Price estimates provided before scheduled procedures
            - Uninsured patient discounts up to 40%
            - No surprise billing policy in compliance with federal regulations
            """,
            "metadata": {"source": "insurance_billing", "category": "billing"}
        },
        {
            "content": """
            Hospital Hours and Contact Information:

            Main Hospital: Open 24/7, 365 days a year
            Emergency Department: Open 24/7, average wait time 15 minutes
            Outpatient Clinics: Monday-Friday 7AM-7PM, Saturday 8AM-4PM, Sunday 10AM-3PM
            Pharmacy: Monday-Friday 8AM-8PM, Saturday 9AM-5PM, Sunday 10AM-2PM
            Laboratory: 24/7 for emergency tests, outpatient hours 6AM-6PM
            Radiology: 24/7 for emergency imaging, scheduled appointments 7AM-9PM

            Contact Information:
            Main Number: (555) 123-4567
            Emergency: 911 or (555) 123-4911
            Appointments: (555) 123-4800
            Billing Questions: (555) 123-4900
            Medical Records: (555) 123-4950
            Patient Relations: (555) 123-4975
            Pharmacy: (555) 123-4820

            Address: 123 Healthcare Boulevard, Medical City, MC 12345
            Website: www.medicityhealth.org
            Patient Portal: portal.medicityhealth.org
            Email: info@medicityhealth.org
            """,
            "metadata": {"source": "contact_hours", "category": "general"}
        },
        {
            "content": """
            Patient Services and Amenities:

            Patient Rooms:
            - Private rooms with private bathrooms
            - Free WiFi throughout the facility
            - Television with premium channels
            - Family sleeping accommodations available
            - Room service meals 6AM-10PM
            - Quiet hours from 9PM-7AM

            Visitor Information:
            - Visiting hours: 8AM-8PM for most units
            - ICU visiting: 10AM-2PM and 4PM-8PM
            - Two visitors per patient at a time
            - Children under 12 must be accompanied by adult
            - Visitor badges required, obtained at main desk
            - Overnight visitor accommodations available for out-of-town families

            Amenities:
            - Gift shop open 9AM-6PM daily
            - Cafeteria open 6AM-10PM, healthy meal options
            - Coffee shop open 5:30AM-8PM
            - Meditation room and chapel open 24/7
            - Business center with computers and printing
            - Valet parking available 7AM-7PM ($5)
            - Self-parking garage ($3/hour, $15/day maximum)
            - ATM machines located on each floor
            """,
            "metadata": {"source": "patient_services", "category": "amenities"}
        },
        {
            "content": """
            Quality and Safety Information:

            Accreditations and Certifications:
            - Joint Commission accreditation with Gold Seal of Approval
            - Magnet Recognition for nursing excellence
            - American Heart Association Get With The Guidelines Gold Plus
            - American College of Surgeons verified Level II Trauma Center
            - Centers for Medicare & Medicaid Services 5-star rating

            Quality Measures:
            - Hospital-acquired infection rates below national average
            - Patient satisfaction scores in top 10% nationally
            - Readmission rates 20% below national average
            - Zero tolerance policy for preventable medical errors
            - Comprehensive patient safety program with dedicated staff

            Technology:
            - Electronic health records system for seamless care coordination
            - Telemedicine services available for follow-up appointments
            - Advanced robotic surgery capabilities
            - AI-powered diagnostic imaging
            - Real-time patient monitoring systems

            Staff Qualifications:
            - Board-certified physicians in all specialties
            - Nursing staff with BSN requirement
            - Continuing education requirements for all clinical staff
            - Multilingual staff and interpretation services available
            """,
            "metadata": {"source": "quality_safety", "category": "quality"}
        },
        {
            "content": """
            Emergency Preparedness and Policies:

            Emergency Procedures:
            - Comprehensive disaster preparedness plan
            - Emergency generator backup power for entire facility
            - Emergency supplies maintained for 72-hour self-sufficiency
            - Regular emergency drills conducted monthly
            - Coordination with local emergency management agencies

            Infection Control:
            - Strict hand hygiene protocols enforced
            - Isolation procedures for infectious diseases
            - Personal protective equipment readily available
            - Air filtration systems with HEPA filters
            - Regular environmental cleaning with hospital-grade disinfectants

            Patient Rights and Responsibilities:
            - Right to respectful and dignified treatment
            - Right to participate in care decisions
            - Right to privacy and confidentiality
            - Right to access medical records
            - Right to interpreter services if needed
            - Responsibility to provide accurate health information
            - Responsibility to follow treatment plans
            - Responsibility to treat staff and other patients with respect

            Privacy Policies:
            - HIPAA compliant privacy practices
            - Secure patient information systems
            - Limited access to patient information on need-to-know basis
            - Patient consent required for information sharing
            """,
            "metadata": {"source": "emergency_policies", "category": "policies"}
        },
        {
            "content": """
            Wellness and Prevention Programs:

            Community Health Programs:
            - Free health screenings: blood pressure, cholesterol, diabetes
            - Annual health fairs with multiple screening options
            - Vaccination clinics including flu, COVID-19, and travel vaccines
            - Smoking cessation programs with counseling and support groups
            - Weight management programs with nutritionist consultations

            Educational Services:
            - Diabetes education and management classes
            - Heart healthy cooking classes
            - CPR and first aid training for community members
            - New parent classes and breastfeeding support
            - Senior health and wellness programs
            - Health lectures by physicians on various topics

            Support Groups:
            - Cancer survivors support group (Tuesdays 7PM)
            - Diabetes management group (Thursdays 6PM)
            - Heart disease support group (First Monday 7PM)
            - Mental health support groups (various times)
            - Grief counseling support groups
            - Family caregiver support groups

            Fitness and Rehabilitation:
            - Cardiac rehabilitation program
            - Pulmonary rehabilitation
            - Physical therapy and occupational therapy
            - Aquatic therapy pool
            - Fitness classes for seniors
            """,
            "metadata": {"source": "wellness_programs", "category": "wellness"}
        },
        {
            "content": """
            Medical Staff and Physician Information:

            Medical Staff Organization:
            - Over 200 board-certified physicians
            - Active medical staff with hospital privileges
            - Consulting physicians available 24/7
            - Residency and fellowship training programs
            - Continuing medical education requirements

            Physician Specialties Available:
            - Anesthesiology, Cardiology, Cardiothoracic Surgery
            - Critical Care Medicine, Dermatology, Emergency Medicine
            - Endocrinology, Family Medicine, Gastroenterology
            - General Surgery, Hematology/Oncology, Infectious Disease
            - Internal Medicine, Nephrology, Neurology, Neurosurgery
            - Obstetrics/Gynecology, Ophthalmology, Orthopedic Surgery
            - Otolaryngology, Pathology, Pediatrics, Physical Medicine
            - Psychiatry, Pulmonology, Radiology, Rheumatology
            - Urology, Vascular Surgery

            Physician Services:
            - Second opinion consultations available
            - Telemedicine appointments for follow-up care
            - Same-day sick visits in family medicine
            - 24/7 physician on-call service for emergencies
            - Physician-led multidisciplinary care teams
            - Care coordination between specialists

            Nursing Excellence:
            - Magnet-designated nursing program
            - Bachelor's degree requirement for all RNs
            - Specialized certification in critical areas
            - Continuing education and professional development
            - Nurse-to-patient ratios meet national standards
            """,
            "metadata": {"source": "medical_staff", "category": "staff"}
        },
        {
            "content": """
            Research and Innovation:

            Clinical Research:
            - Active participation in clinical trials for cancer, heart disease, and diabetes
            - Research partnerships with major universities and medical centers
            - IRB-approved research protocols ensuring patient safety
            - Opportunities for patients to participate in cutting-edge treatments
            - Research coordinator available to discuss trial options

            Medical Innovation:
            - Latest medical technologies and equipment
            - Robotic surgery programs in multiple specialties
            - Artificial intelligence integration in diagnostic imaging
            - Electronic health records with predictive analytics
            - Telemedicine and remote monitoring capabilities

            Quality Improvement Initiatives:
            - Continuous quality improvement programs
            - Patient safety initiatives and error reduction protocols
            - Staff engagement in quality improvement projects
            - Patient and family advisory councils
            - Performance measurement and public reporting

            Educational Partnerships:
            - Medical residency and fellowship programs
            - Nursing clinical rotation partnerships
            - Allied health professional training programs
            - Medical student rotations
            - Continuing education for healthcare professionals

            Community Partnerships:
            - Collaboration with local schools for health education
            - Partnership with community organizations for health fairs
            - Mobile health screening units for underserved areas
            - Workplace wellness programs for local employers
            """,
            "metadata": {"source": "research_innovation", "category": "innovation"}
        }
    ]

    documents = []
    for doc in hospital_docs:
        documents.append(Document(
            page_content=doc["content"],
            metadata=doc["metadata"]
        ))

    return documents

def setup_rag_pipeline():
    """Set up the RAG pipeline with embeddings and vector store."""
    documents = create_knowledge_base()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    splits = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(
        model_name="all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )

    vectorstore = FAISS.from_documents(splits, embeddings)
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    return retriever

@tool
def search_knowledge_base(query: str) -> str:
    """Search the hospital knowledge base for information about services, policies, or general hospital info."""
    try:
        retriever = setup_rag_pipeline()
        docs = retriever.get_relevant_documents(query)

        if not docs:
            return "I couldn't find specific information about your query in our knowledge base."

        context = "\n\n".join([doc.page_content for doc in docs])
        response = f"Based on our hospital information:\n\n{context}"

        return response
    except Exception as e:
        return f"I encountered an error while searching: {str(e)}"


In [12]:
# ============================================================================
# STEP 5: SIMPLE HEALTHCARE CHATBOT WITH DIRECT TOOL CALLING
# ============================================================================
from google.colab import userdata
api = userdata.get('Gemini_Api_Key')

class HealthcareChatbot:
    def __init__(self):
        """Initialize the healthcare chatbot with Gemini 2.0 and tools."""

        # Initialize Gemini 2.0 Flash
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-exp",
            google_api_key=api,
            temperature=0.1
        )

        # Define tools
        self.tools = [
            search_appointments,
            book_appointment,
            update_appointment,
            cancel_appointment,
            search_doctors,
            get_patient_info,
            search_knowledge_base
        ]

        # Bind tools to LLM
        self.llm_with_tools = self.llm.bind_tools(self.tools)

        # System message
        self.system_message = SystemMessage(content="""
        You are a helpful healthcare assistant at Medical City Hospital.

        You can help patients with:
        1. **Appointment Management**: Search, book, update, or cancel appointments
        2. **Doctor Search**: Find doctors by specialization or name
        3. **Hospital Information**: Hours, services, insurance, billing, and policies
        4. **Patient Information**: Look up patient details and appointment history

        **Important Instructions:**
        - Always use the appropriate tools to get accurate, real-time information
        - For appointments: Ask for patient ID, doctor ID, dates (YYYY-MM-DD), times (HH:MM)
        - Be professional, empathetic, and helpful
        - Provide clear confirmations for all actions
        - If information is missing, politely ask for the specific details needed
        - Always search the knowledge base for hospital information queries

        **Available Tools:**
        - search_appointments: Find existing appointments
        - book_appointment: Schedule new appointments
        - update_appointment: Modify existing appointments
        - cancel_appointment: Cancel appointments
        - search_doctors: Find doctors by specialty/name
        - get_patient_info: Get patient details and history
        - search_knowledge_base: Get hospital information

        Respond naturally and helpfully to user queries.
        """)

        # Initialize conversation history
        self.conversation_history = [self.system_message]

        # Setup database
        create_database()
        populate_sample_data()
        setup_rag_pipeline()

        print("Healthcare Chatbot initialized successfully!")
        print("Database ready with sample data.")
        print("Ready to help with appointments, doctor search, and hospital information!")

    def chat(self, user_message: str) -> str:
        """Chat with the healthcare assistant."""
        try:
            # Add user message to conversation
            user_msg = HumanMessage(content=user_message)
            self.conversation_history.append(user_msg)

            # Get response from LLM with tools
            response = self.llm_with_tools.invoke(self.conversation_history)

            # Handle tool calls if present
            if hasattr(response, 'tool_calls') and response.tool_calls:
                # Add the assistant's response with tool calls
                self.conversation_history.append(response)

                # Execute tool calls
                tool_results = []
                for tool_call in response.tool_calls:
                    # Find and execute the tool
                    tool_name = tool_call['name']
                    tool_args = tool_call['args']

                    # Execute the tool
                    for tool in self.tools:
                        if tool.name == tool_name:
                            result = tool.invoke(tool_args)
                            tool_results.append({
                                'tool_call_id': tool_call['id'],
                                'name': tool_name,
                                'result': result
                            })
                            break

                # Create tool messages
                from langchain_core.messages import ToolMessage
                tool_messages = []
                for tr in tool_results:
                    tool_messages.append(
                        ToolMessage(
                            content=str(tr['result']),
                            tool_call_id=tr['tool_call_id']
                        )
                    )

                # Add tool messages to conversation
                self.conversation_history.extend(tool_messages)

                # Get final response from LLM
                final_response = self.llm_with_tools.invoke(self.conversation_history)
                self.conversation_history.append(final_response)

                return final_response.content

            else:
                # No tools needed, just return the response
                self.conversation_history.append(response)
                return response.content

        except Exception as e:
            error_msg = f"I encountered an error: {str(e)}. Please try again or rephrase your question."
            return error_msg

    def reset_conversation(self):
        """Reset the conversation history."""
        self.conversation_history = [self.system_message]
        print("Conversation reset!")

    def get_conversation_history(self):
        """Get the current conversation history."""
        return self.conversation_history


In [14]:
chatbot = HealthcareChatbot()
chatbot.chat("I want to book an appointment")

Database created successfully!
Comprehensive sample data populated successfully!
📊 Data Summary:
  • 20 Patients
  • 15 Doctors across 15 specializations
  • 25 Hospital services
  • 30 Appointments (past, current, and future)


/tmp/ipython-input-2283767018.py:353: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Healthcare Chatbot initialized successfully!
Database ready with sample data.
Ready to help with appointments, doctor search, and hospital information!


"Okay! I can help you with that. I'll need a bit more information to book the appointment. Can you please provide the following:\n\n*   **Patient ID:**\n*   **Doctor ID:**\n*   **Date of Appointment:** (YYYY-MM-DD)\n*   **Time of Appointment:** (HH:MM)\n*   **Reason for Appointment:**"

In [28]:
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

cursor.execute("SELECT * FROM patients")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

(1, 'John', 'Doe', 'john.doe@email.com', '+1234567890', '1985-05-15', 'Blue Cross Blue Shield', '2025-09-15 20:55:00')
(2, 'Jane', 'Smith', 'jane.smith@email.com', '+1234567891', '1990-08-22', 'Aetna', '2025-09-15 20:55:00')
(3, 'Mike', 'Johnson', 'mike.johnson@email.com', '+1234567892', '1978-12-10', 'Medicare', '2025-09-15 20:55:00')
(4, 'Sarah', 'Wilson', 'sarah.wilson@email.com', '+1234567893', '1995-03-08', 'Cigna', '2025-09-15 20:55:00')
(5, 'David', 'Brown', 'david.brown@email.com', '+1234567894', '1982-11-30', 'United Healthcare', '2025-09-15 20:55:00')
(6, 'Emma', 'Davis', 'emma.davis@email.com', '+1234567895', '1988-09-14', 'Blue Cross Blue Shield', '2025-09-15 20:55:00')
(7, 'Chris', 'Miller', 'chris.miller@email.com', '+1234567896', '1975-07-02', 'Medicare', '2025-09-15 20:55:00')
(8, 'Lisa', 'Garcia', 'lisa.garcia@email.com', '+1234567897', '1992-04-18', 'Aetna', '2025-09-15 20:55:00')
(9, 'Robert', 'Martinez', 'robert.martinez@email.com', '+1234567898', '1980-12-25', 'Cig

In [15]:
!pip install gradio

In [16]:
import gradio as gr

In [30]:
chatbot = HealthcareChatbot()

def respond(message, history):
    """Handles user input and returns chatbot response, updating history."""
    # history is a list of tuples, [(user_msg, bot_msg), ...]
    # The HealthcareChatbot expects a list of BaseMessage objects

    # Get the response from the chatbot
    response = chatbot.chat(message) # The chat method handles history internally
    return response

Database created successfully!
Comprehensive sample data populated successfully!
📊 Data Summary:
  • 20 Patients
  • 15 Doctors across 15 specializations
  • 25 Hospital services
  • 30 Appointments (past, current, and future)
Healthcare Chatbot initialized successfully!
Database ready with sample data.
Ready to help with appointments, doctor search, and hospital information!


In [31]:
iface = gr.ChatInterface(fn=respond, title="Medical City Healthcare Chatbot")

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [32]:
iface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://70342d4bb9d84be3d0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/tmp/ipython-input-2283767018.py:368: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://70342d4bb9d84be3d0.gradio.live
